```c++
#include <iostream>
#include <cmath>
#include <map>

// silnia

template<typename T>
T factorial(T x) {
    T p = 1;
    for (int i = 0; i < x; ++i) {
        p *= (i + 1);
    }
    return p;
}

// licznik i-tego wyrazu szeregu Taylora

template<typename T>
T sin_term_nominator(int i, T x) {
    T s = i % 2 ? -1 : 1;
    T a = pow(x, 2 * i + 1);
    return s * a;
}

// mianownik i-tego wyrazu szeregu Taylora

template<typename T>
T sin_term_denominator(int i, T x) {
    T b = factorial(2 * i + 1);
    return b;
}

// i-ty wyraz szeregu Taylora

template<typename T>
T sin_term(int i, T x) {
    T n = sin_term_nominator(i, x);
    T d = sin_term_denominator(i, x);
    return n / d;
}

// sinus wyliczany poprzez sumowanie wyrazów szeregu Taylora od największego do najmniejszego

template<typename T>
T sin1_ltr(T x, int n) {
    T s = 0;
    for(int i = 0; i < n; ++i) {
        s += sin_term<T>(i, x);
    }
    return s;
}

// sinus wyliczany poprzez sumowanie wyrazów szeregu Taylora od najmniejszego do największego

template<typename T>
T sin1_rtl(T x, int n) {
    T s = 0;
    for(int i = n - 1; i >= 0; --i) {
        s += sin_term<T>(i, x);
    }
    return s;
}

// sinus wyliczany poprzez sumowanie wyrazów szeregu Taylora, gdzie każdy wyraz jest wyliczany na podstawie poprzedniego

template<typename T>
T sin2(T x, int n) {
    T s = x;
    T y = x;
    T z = 1;
    for(int i = 1; i < n; ++i) {
        y *= -x * x;
        z *= (2 * i) * (2 * i + 1);
        s += y / z;
    }
    return s;
}

// funkcja główna, wypisuje wartość sinusa przyjmując metodę, typ zmiennoprzecinkowy, liczbę wyrazów szeregu oraz x
// przykład: ./sin sin2 double 8 0.5

int main(int argc, const char * argv[]) {
    int prec = std::numeric_limits<long double>::max_digits10;
    std::cout.precision(prec);
    std::cout << std::fixed;
    
    std::map<std::string, std::map<std::string, std::function<long double(long double, int)>>> f;

    f["sin1_ltr"]["float"] = sin1_ltr<float>;
    f["sin1_ltr"]["double"] = sin1_ltr<double>;
    f["sin1_ltr"]["longdouble"] = sin1_ltr<long double>;
    
    f["sin1_rtl"]["float"] = sin1_rtl<float>;
    f["sin1_rtl"]["double"] = sin1_rtl<double>;
    f["sin1_rtl"]["longdouble"] = sin1_rtl<long double>;
    
    f["sin2"]["float"] = sin2<float>;
    f["sin2"]["double"] = sin2<double>;
    f["sin2"]["longdouble"] = sin2<long double>;

    std::string fn = argv[1];
    std::string tp = argv[2];
    int n = std::stoi(argv[3]);
    long double x = std::stold(argv[4]);
    
    auto sin_f = f[fn][tp];
    
    long double v = sin_f(x, n);
    long double sv = std::sin(x);
    
    std::cout << std::abs(v - sv) / sv;
    
    return 0;
}

```

In [31]:
from metody_numeryczne_projekt1 import *

In [32]:
plot_re_x(8)

Powyższy wykres przedstawia zależność błędu względnego od wartości `x`, dla sumy 8 pierwszych wyrazów szeregu Taylora. Oś y jest w skali logarytmicznej. Błąd względny wyliczany jest w odniesieniu do najdokładniejszej funkcji bibliotecznej (`std::sinl`). Widać wyraźnie, że największy wpływ na dokładność wyniku ma zastosowany typ danych. Charakterystycznym elementem wykresu jest strome zbocze składające się z kilku nałożonych krzywych. Jest to przedział, w którym 8 wyrazów ciągu nie jest wystarczającym zaokrągleniem. Po przekroczeniu punktu krytycznego, błąd rośnie wykładniczo, proporcjonalnie do `x`. Zależność błędu względnego od ilości wyliczonych wyrazów szeregu będzie omówiona w dalszej części sprawozdania.

In [33]:
plot_re_x(16)

Po zwiększeniu obliczanej liczby wyrazów do 16, krzywe stabilizują sie. Wyraźnie widać zgrupowane funkcje, których wspólną cechą jest zastosowany typ danych. Przerwy w ciągłości w dolnym obszarze wykresu są spowodowane faktem, iż dla typu longdouble, błąd względny dla niektórych argumentów wynosi 0. Z racji zastosowania skali logarytmicznej, zera nie da się reprezentować.

In [41]:
plot_re_x(32)

In [35]:
plot_re_x_lin('float', 16)

Powyższy wykres błędu względnego od `x`, dla typu `float` oraz 16 wyrazów szeregu, tym razem z osią Y w skali liniowej, wyraźnie pokazuje różnice pomiędzy zastosowanymi metodami. Najlepsze przybliżenie daje funkcja `sin1_rtl`, sumująca wyrazy od najmniejszego do największego. Jest to spowodowane faktem, iż przy sumowaniu w kolejności przeciwnej, pewna liczba ostatnich wyrazów szeregu w ogóle nie jest dodawana do akumulatora, gdyż są zbyt małe. `sin2` (wyliczająca każdy wyraz na bazie poprzedniego) prawie zawsze pokrywa się z `sin1_ltr` (sumującą wyrazy od największego do najmniejszego).

In [36]:
plot_re_x_lin('double', 16)

Poza o kilka rzędów wilekości mniejszymi wartościami błędu względnego, analogiczny wykres dla typu double nie różni się znacząco od wykresu dla typu float.

In [37]:
plot_re_x_lin('longdouble', 16)

Wykres dla typu longdouble również nie różni się charakterystyką od poprzednich wykresów, aczkolwiek warto zwrócić uwagę, iż funkcja `sin1_rtl` (sumująca wyrazy od najmniejszego do największego) dla prawie wszystkich `x`-ów zwróciła wartość równą tej zwróconej przez funkcję biblioteczną `std::sinl`. Pojedyncze wyjątki mogą być spowodowane zastosowaniem przez funkcję biblioteczną 80- lub 128-bitowych rejestrów zmiennoprzecinkowych.

In [38]:
plot_re_n(0.1)

In [39]:
plot_re_n(0.9)

In [40]:
plot_re_n(1.45)